In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
from functions import *

In [10]:
input_noise = 2
n_models = 30
n_metrics = 4

In [11]:
os.listdir('./results/')

['Beer']

In [12]:
path = './results/'
name = 'Beer/'

In [14]:
lnl = pd.read_csv(path + name + 'lnl.csv')
model1 = pd.read_csv(path + name + 'model1.csv')
model2 = pd.read_csv(path + name + 'model2.csv')
rbf = pd.read_csv(path + name + 'rbf.csv')
ffann = pd.read_csv(path + name + 'ffann.csv')
others = pd.read_csv(path + name + 'other.csv')
y = others['Y_true'].values
sarima = others['SARIMA'].values
wmes = others['WMES'].values

In [15]:
lnl_metrics = np.array([[RMSE(y[input_noise + 1:], lnl.iloc[input_noise + 1:, i]) for i in range(n_models)],
                [MAPE(y[input_noise + 1:], lnl.iloc[input_noise + 1:, i]) for i in range(n_models)],
                [MdAPE(y[input_noise + 1:], lnl.iloc[input_noise + 1:, i]) for i in range(n_models)],
                [DA(y[input_noise + 1:], lnl.iloc[input_noise + 1:, i]) for i in range(n_models)]])

model1_metrics = np.array([[RMSE(y[input_noise + 1:], model1.iloc[:, i]) for i in range(n_models)],
                [MAPE(y[input_noise + 1:], model1.iloc[:, i]) for i in range(n_models)],
                [MdAPE(y[input_noise + 1:], model1.iloc[:, i]) for i in range(n_models)],
                [DA(y[input_noise + 1:], model1.iloc[:, i]) for i in range(n_models)]])

model2_metrics = np.array([[RMSE(y[input_noise + 1:], model2.iloc[input_noise:, i]) for i in range(n_models)],
                [MAPE(y[input_noise + 1:], model2.iloc[input_noise:, i]) for i in range(n_models)],
                [MdAPE(y[input_noise + 1:], model2.iloc[input_noise:, i]) for i in range(n_models)],
                [DA(y[input_noise + 1:], model2.iloc[input_noise:, i]) for i in range(n_models)]])

ffann_metrics = np.array([[RMSE(y, ffann.iloc[:, i]) for i in range(n_models)],
                [MAPE(y, ffann.iloc[:, i]) for i in range(n_models)],
                [MdAPE(y, ffann.iloc[:, i]) for i in range(n_models)],
                [DA(y, ffann.iloc[:, i]) for i in range(n_models)]])

rbf_metrics = np.array([[RMSE(y, rbf.iloc[:, i]) for i in range(n_models)],
                [MAPE(y, rbf.iloc[:, i]) for i in range(n_models)],
                [MdAPE(y, rbf.iloc[:, i]) for i in range(n_models)],
                [DA(y, rbf.iloc[:, i]) for i in range(n_models)]])

sarima_metrics = np.array([RMSE(y, sarima), MAPE(y, sarima), MdAPE(y, sarima), DA(y, sarima)])

wmes_metrics = np.array([RMSE(y, wmes), MAPE(y, wmes), MdAPE(y, wmes), DA(y, wmes)])

In [16]:
metrics = [ffann_metrics,
           rbf_metrics,
           lnl_metrics,
           model1_metrics,
           model2_metrics]

model_names = ['FFANN', 'RBF', 'L&NL ANN', 'Model 1', 'Model 2']
df_metrics = pd.DataFrame(data= np.zeros((4, len(metrics) + 2)), index = ['RMSE', 'MAPE', 'MdAPE', 'DA'],
                          columns= ['SARIMA', 'WMES'] + model_names)
df_metrics['SARIMA'] = sarima_metrics
df_metrics['WMES'] = wmes_metrics

# Mean

In [17]:
for i, model_name in enumerate(model_names):
    df_metrics[model_name] = np.mean(metrics[i], axis = 1)
df_metrics

,SARIMA,WMES,FFANN,RBF,L&NL ANN,Model 1,Model 2
RMSE,0.153339,0.0,0.161277,0.114311,0.355798,0.319626,0.119180
MAPE,0.147982,0.0,0.217652,0.135739,0.255313,0.254792,0.160511
MdAPE,0.087893,0.0,0.198165,0.122787,0.146354,0.145344,0.139695
DA,0.789474,0.0,0.345614,0.619298,0.568203,0.533333,0.575558


# Median

In [18]:
for i, model_name in enumerate(model_names):
    df_metrics[model_name] = np.median(metrics[i], axis = 1)
df_metrics

,SARIMA,WMES,FFANN,RBF,L&NL ANN,Model 1,Model 2
RMSE,0.153339,0.0,0.151986,0.107388,0.127533,0.103245,0.120705
MAPE,0.147982,0.0,0.199685,0.127235,0.165169,0.138650,0.150751
MdAPE,0.087893,0.0,0.180278,0.115558,0.127000,0.139018,0.129896
DA,0.789474,0.0,0.256579,0.526316,0.647059,0.625000,0.647059


# Minimum

In [20]:
for i, model_name in enumerate(model_names):
    df_metrics[model_name] = np.min(metrics[i], axis = 1)
df_metrics

,SARIMA,WMES,FFANN,RBF,L&NL ANN,Model 1,Model 2
RMSE,0.153339,0.0,0.058296,0.068430,0.063102,0.062604,0.068094
MAPE,0.147982,0.0,0.063102,0.079355,0.080736,0.080043,0.087867
MdAPE,0.087893,0.0,0.047308,0.073746,0.062230,0.053509,0.061869
DA,0.789474,0.0,-0.100000,0.210526,0.052632,0.000000,0.111111


# Statistical test (Friedman)

In [21]:
from scipy.stats import friedmanchisquare

In [22]:
friedmanchisquare(lnl_metrics[0, :], model1_metrics[0, :], model2_metrics[0, :])

FriedmanchisquareResult(statistic=0.46666666666664014, pvalue=0.7918895663367922)